# Assignment 6

Develop RNN model in pytorch to solve the following problem:  
    
1. Detect sarcasm 
Data from https://www.kaggle.com/sherinclaudia/sarcastic-comments-on-reddit  
Your quality metric = accuracy  
Randomly select 20% of your data for test set. You can use it only for final perfomance estimation.   
 

Remember, you can use GPU resourses in kaggle kernels.

In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
import nltk
import gensim
import spacy
from tqdm import tqdm_notebook

from sklearn import metrics

import torch as tt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext.data import Field, LabelField, BucketIterator, TabularDataset, Iterator



SEED = 42
np.random.seed(SEED)

D:\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)
D:\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def tokenizer(text): 
    res = [lemmatizer.lemmatize(x) for x in word_tokenize(text)
            if x and x not in stopwords]
    return res

In [9]:
df = pd.read_csv('sarcasm.csv')
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [10]:
len(df)

1010826

In [14]:
# df1 = pd.concat([df['comment'], df['label']], axis=1)
df1 = df[['comment', 'label']].copy()
df1.to_csv('sarcasm1.csv', index=False)
# df1

In [15]:
TEXT = Field(include_lengths=True, batch_first=True, 
             tokenize=tokenizer,
             eos_token='<eos>',
             lower=True,
             stop_words=stopwords)
LABEL = LabelField(dtype=tt.int64, use_vocab=True)

df = TabularDataset('sarcasm1.csv', format='csv', 
                         fields=[('comment', TEXT), ('label', LABEL)], 
                         skip_header=True)

In [16]:
TEXT.build_vocab(df, min_freq=5, vectors="glove.6B.100d")
len(TEXT.vocab.itos)

41043

In [17]:
TEXT.vocab.itos[:10]

['<unk>', '<pad>', '<eos>', '.', ',', "'s", "n't", '?', '!', '...']

In [18]:
train, test = df.split(0.8, stratified=True)

In [19]:
train, valid = train.split(0.9, stratified=True)

In [20]:
LABEL.build_vocab(df)

In [21]:
weights_matrix = TEXT.vocab.vectors

In [22]:
LABEL.vocab.itos[:3]

['0', '1']

In [23]:
class MyModel(nn.Module):
    
    def __init__(self, vocab_size, embed_size, hidden_size, weights_matrix):
        super(MyModel, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_size)
        self.embedding = nn.Embedding.from_pretrained(weights_matrix, freeze=True)
        
        self.rnn = nn.LSTM(input_size=embed_size,
                           hidden_size=hidden_size,
                           bidirectional=True,
                           batch_first=True,
                          )
        
        self.fc = nn.Linear(hidden_size * 2 *2, 3)
        
    def forward(self, batch):
        
        x, x_lengths = batch.comment
        x = x.to(device)
        batch.label = batch.label.to(device)
        x = self.embedding(x)

        if x_lengths is not None:
            x_lengths = x_lengths.view(-1).tolist()
            x = nn.utils.rnn.pack_padded_sequence(x, x_lengths, batch_first=True)
            
        _, (hidden, cell) = self.rnn(x)
        
        hidden = hidden.transpose(0,1)
        cell = cell.transpose(0,1)
        hidden = hidden.contiguous().view(hidden.size(0),-1)
        cell = cell.contiguous().view(cell.size(0),-1)
        x = tt.cat([hidden, cell], dim=1).squeeze(1)
        x = self.fc(x)
        return x

In [24]:
batch_size = 32
embed_size=100
hidden_size=128
device = tt.device('cuda:0' if tt.cuda.is_available() else 'cpu')

model = MyModel(len(TEXT.vocab.itos),
                embed_size,
                hidden_size,
                weights_matrix)

model = model.to(device)

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, valid, test),
    batch_sizes=(batch_size, batch_size, batch_size),
    shuffle=True,
    sort_key=lambda x: len(x.comment),
    sort_within_batch=True,
)

optimizer = optim.Adam(model.parameters())
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True, cooldown=5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)

In [25]:
from sklearn.metrics import accuracy_score
def acc(model, iterator):
    model.eval()
    acc_ = 0

    n_batches = len(iterator)
    with tt.no_grad():
        for batch in iterator:
            pred = tt.nn.functional.softmax(model(batch), dim=1).cpu().numpy().argmax(axis=1)
            true = batch.label.cpu()
            accuracy = accuracy_score(pred, true)
            acc_ += accuracy

    return acc_ / n_batches


In [26]:
def _train_epoch(model, iterator, optimizer, criterion, curr_epoch):

    model.train()

    running_loss = 0

    n_batches = len(iterator)
    iterator = tqdm_notebook(iterator, total=n_batches, desc='epoch %d' % (curr_epoch), leave=True)

    for i, batch in enumerate(iterator):
        optimizer.zero_grad()

        pred = model(batch)
        loss = criterion(pred, batch.label)
        loss.backward()
        optimizer.step()

        curr_loss = loss.data.cpu().detach().item()
        
        loss_smoothing = i / (i+1)
        running_loss = loss_smoothing * running_loss + (1 - loss_smoothing) * curr_loss

        iterator.set_postfix(loss='%.5f' % running_loss)

    return running_loss

def _test_epoch(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    n_batches = len(iterator)
    with tt.no_grad():
        for batch in iterator:
            pred = model(batch)
            loss = criterion(pred, batch.label)
            epoch_loss += loss.data.item()

    return epoch_loss / n_batches

In [27]:
criterion = nn.CrossEntropyLoss()
def nn_train(model, train_iterator, valid_iterator, criterion, optimizer, n_epochs=100,
          scheduler=None, early_stopping=0):

    prev_loss = 100500
    es_epochs = 0
    best_epoch = None
    history = pd.DataFrame()

    for epoch in range(n_epochs):
        train_loss = _train_epoch(model, train_iterator, optimizer, criterion, epoch)
        valid_loss = _test_epoch(model, valid_iterator, criterion)

        valid_loss = valid_loss
        print('validation loss %.5f' % valid_loss)

        record = {'epoch': epoch, 'train_loss': train_loss, 'valid_loss': valid_loss}
        history = history.append(record, ignore_index=True)

        if early_stopping > 0:
            if valid_loss > prev_loss:
                es_epochs += 1
            else:
                es_epochs = 0

            if es_epochs >= early_stopping:
                best_epoch = history[history.valid_loss == history.valid_loss.min()].iloc[0]
                print('Early stopping! best epoch: %d val %.5f' % (best_epoch['epoch'], best_epoch['valid_loss']))
                break

            prev_loss = min(prev_loss, valid_loss)

In [28]:
nn_train(model, train_iterator, valid_iterator, criterion, optimizer, scheduler=scheduler, 
        n_epochs=10, early_stopping=2)

HBox(children=(IntProgress(value=0, description='epoch 0', max=22744, style=ProgressStyle(description_width='initial')), HTML(value='')))


validation loss 0.56327


HBox(children=(IntProgress(value=0, description='epoch 1', max=22744, style=ProgressStyle(description_width='initial')), HTML(value='')))


validation loss 0.55510


HBox(children=(IntProgress(value=0, description='epoch 2', max=22744, style=ProgressStyle(description_width='initial')), HTML(value='')))


validation loss 0.55422


HBox(children=(IntProgress(value=0, description='epoch 3', max=22744, style=ProgressStyle(description_width='initial')), HTML(value='')))


validation loss 0.55772


HBox(children=(IntProgress(value=0, description='epoch 4', max=22744, style=ProgressStyle(description_width='initial')), HTML(value='')))


validation loss 0.56555
Early stopping! best epoch: 2 val 0.55422


In [29]:
acc(model, test_iterator)

0.7066601736740625